#Named Entity Recognition Fine Tuned

In [ ]:
! pip install datasets transformers seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 93.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 69.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 k

In [ ]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')

# load dataset cleaned
cleaned_data_path = "drive/MyDrive/WhereOrderBot/data/ner-dataset-cleaned.xlsx"
ner_dataset = pd.read_excel(cleaned_data_path)
ner_dataset.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,category,question,tokens,ner_tags
0,where_is_my_order,I have order with #123412 (123123) AASDAS #ASDAS,"['I', 'have', 'order', 'with', '#123412', '(12...","[0, 0, 0, 0, 1, 1, 1, 1]"
1,where_is_my_order,"Hello, just checking in on the status of my or...","['Hello,', 'just', 'checking', 'in', 'on', 'th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,where_is_my_order,It's getting frustrating not knowing where my ...,"[""It's"", 'getting', 'frustrating', 'not', 'kno...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,where_is_my_order,"Hey, I recently ordered a custom mechanical ke...","['Hey,', 'I', 'recently', 'ordered', 'a', 'cus...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,where_is_my_order,"Hello, I haven't received any updates regardin...","['Hello,', 'I', ""haven't"", 'received', 'any', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."


In [ ]:
ner_dataset['tokens'][0]

"['I', 'have', 'order', 'with', '#123412', '(123123)', 'AASDAS', '#ASDAS']"

In [ ]:
import ast
# Convert the string to a list
result_list = ast.literal_eval(ner_dataset['tokens'][0])
result_list

['I', 'have', 'order', 'with', '#123412', '(123123)', 'AASDAS', '#ASDAS']

In [ ]:
# convert to list
from io import StringIO
import json
import ast

def convert_to_list(string_data)->list:
  return ast.literal_eval(string_data)



In [ ]:
ner_dataset['tokens_list'] = ner_dataset['tokens'].apply(lambda tokens: convert_to_list(tokens))
ner_dataset['ner_tags_list'] = ner_dataset['ner_tags'].apply(lambda ner_tags: convert_to_list(ner_tags))
ner_dataset

,category,question,tokens,ner_tags,tokens_list,ner_tags_list
0,where_is_my_order,I have order with #123412 (123123) AASDAS #ASDAS,"['I', 'have', 'order', 'with', '#123412', '(12...","[0, 0, 0, 0, 1, 1, 1, 1]","[I, have, order, with, #123412, (123123), AASD...","[0, 0, 0, 0, 1, 1, 1, 1]"
1,where_is_my_order,"Hello, just checking in on the status of my or...","['Hello,', 'just', 'checking', 'in', 'on', 'th...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Hello,, just, checking, in, on, the, status, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,where_is_my_order,It's getting frustrating not knowing where my ...,"[""It's"", 'getting', 'frustrating', 'not', 'kno...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[It's, getting, frustrating, not, knowing, whe...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,where_is_my_order,"Hey, I recently ordered a custom mechanical ke...","['Hey,', 'I', 'recently', 'ordered', 'a', 'cus...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Hey,, I, recently, ordered, a, custom, mechan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,where_is_my_order,"Hello, I haven't received any updates regardin...","['Hello,', 'I', ""haven't"", 'received', 'any', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...","[Hello,, I, haven't, received, any, updates, r...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
...,...,...,...,...,...,...
162,where_is_my_order,"Dear Support,\nI ordered a custom mech keyboar...","['Dear', 'Support,\\nI', 'ordered', 'a', 'cust...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[Dear, Support,\nI, ordered, a, custom, mech, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
163,where_is_my_order,"Hi, I ordered a custom mechanical keyboard fro...","['Hi,', 'I', 'ordered', 'a', 'custom', 'mechan...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Hi,, I, ordered, a, custom, mechanical, keybo...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
164,where_is_my_order,"Dear Customer Support Team,\n\nI have recently...","['Dear', 'Customer', 'Support', 'Team,', 'I', ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[Dear, Customer, Support, Team,, I, have, rece...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
165,where_is_my_order,"Hello, just wondering when I should expect my ...","['Hello,', 'just', 'wondering', 'when', 'I', '...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[Hello,, just, wondering, when, I, should, exp...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [ ]:
# split dataset
import numpy as np
seed_number=1
np.random.seed(seed_number) # set same seed for keeping same result

# split train 90%, test 10%
df_train, df_test = np.split(ner_dataset.sample(frac=1, random_state=seed_number),
                                     [ int(.9*len(ner_dataset))])

print(len(df_train), len(df_test))

150 17


In [ ]:
from datasets import Dataset, load_metric

ner_dataset_train = Dataset.from_pandas(df_train)
ner_dataset_test = Dataset.from_pandas(df_test)
ner_dataset_train

Dataset({
    features: ['category', 'question', 'tokens', 'ner_tags', 'tokens_list', 'ner_tags_list', '__index_level_0__'],
    num_rows: 150
})

In [ ]:
from transformers import AutoTokenizer

task = "ner" # Should be one of "ner", "pos" or "chunk"
model_checkpoint = "distilbert-base-uncased"
batch_size = 4
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [ ]:
example = ner_dataset_train[0]
example["tokens_list"]

['Hi',
 'there,',
 'I',
 'placed',
 'an',
 'order',
 'for',
 'a',
 'custom',
 'mechanical',
 'keyboard',
 'a',
 'few',
 'weeks',
 'ago,',
 'but',
 'I',
 "haven't",
 'received',
 'any',
 'updates',
 'about',
 'shipping',
 'or',
 'tracking.',
 'My',
 'order',
 'number',
 'is',
 '#949495.',
 'Can',
 'you',
 'please',
 'give',
 'me',
 'an',
 'update',
 'on',
 'the',
 'status',
 'of',
 'my',
 'order?',
 'Thank',
 'you.']

In [ ]:
example["ner_tags_list"]

[0, 0, 0, 0, 1, 1, 1, 1]

In [ ]:
tokenized_input = tokenizer(example["tokens_list"], is_split_into_words=True)
tokenized_input

{'input_ids': [101, 7592, 1010, 1045, 4033, 1005, 1056, 2363, 2151, 14409, 4953, 1996, 4358, 7829, 3058, 2005, 2026, 2344, 1006, 1001, 11502, 2581, 2683, 2475, 1007, 1012, 2064, 2017, 3531, 2292, 2033, 2113, 2043, 1045, 2064, 5987, 2026, 7661, 6228, 9019, 2000, 7180, 1029, 4067, 2017, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
tokenizer(['Hello,', 'I', "haven't", 'received', 'any', 'updates', 'regarding', 'the', 'estimated', 'shipping', 'date', 'for', 'my', 'order', '(#135792).', 'Can', 'you', 'please', 'let', 'me', 'know', 'when', 'I', 'can', 'expect', 'my', 'custom', 'mechanical', 'keyboard', 'to', 'arrive?', 'Thank', 'you!'], is_split_into_words=True)

{'input_ids': [101, 7592, 1010, 1045, 4033, 1005, 1056, 2363, 2151, 14409, 4953, 1996, 4358, 7829, 3058, 2005, 2026, 2344, 1006, 1001, 11502, 2581, 2683, 2475, 1007, 1012, 2064, 2017, 3531, 2292, 2033, 2113, 2043, 1045, 2064, 5987, 2026, 7661, 6228, 9019, 2000, 7180, 1029, 4067, 2017, 999, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
label_all_tokens = True
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens_list"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples[f"{task}_tags_list"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            # Special tokens have a word id that is None. We set the label to -100 so they are automatically
            # ignored in the loss function.
            if word_idx is None:
                label_ids.append(-100)
            # We set the label for the first token of each word.
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            else:
                label_ids.append(label[word_idx] if label_all_tokens else -100)
            previous_word_idx = word_idx

        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
tokenize_and_align_labels(ner_dataset_train[:5])

{'input_ids': [[101, 7632, 2045, 1010, 1045, 2872, 2019, 2344, 2005, 1037, 7661, 6228, 9019, 1037, 2261, 3134, 3283, 1010, 2021, 1045, 4033, 1005, 1056, 2363, 2151, 14409, 2055, 7829, 2030, 9651, 1012, 2026, 2344, 2193, 2003, 1001, 6365, 2683, 26224, 2629, 1012, 2064, 2017, 3531, 2507, 2033, 2019, 10651, 2006, 1996, 3570, 1997, 2026, 2344, 1029, 4067, 2017, 1012, 102], [101, 7632, 2045, 1010, 3246, 2023, 10373, 4858, 2017, 2092, 2076, 2122, 2524, 2335, 1012, 1012, 1012, 1996, 3114, 1045, 1005, 1049, 6016, 2023, 4471, 2651, 2003, 2138, 1045, 1005, 2310, 4384, 2008, 2130, 2044, 2028, 3204, 2144, 6885, 2026, 2344, 8440, 1005, 1056, 2042, 12057, 2664, 999, 999, 999, 2013, 1996, 2154, 1045, 2081, 7909, 1010, 1045, 2001, 8074, 3180, 1004, 2039, 16616, 12247, 4953, 2026, 2344, 5082, 1998, 4033, 1005, 1056, 2363, 2309, 4807, 2013, 2017, 1012, 1045, 2342, 2115, 2393, 1999, 23218, 2041, 2073, 2026, 7427, 2003, 1010, 4358, 2051, 2005, 5508, 2030, 2151, 2060, 2592, 2008, 2064, 2022, 4207, 2007, 20

In [ ]:
label_list=['O','ORDER_ID']

In [ ]:
tokenized_dataset_train = ner_dataset_train.map(tokenize_and_align_labels, batched=True)
tokenized_dataset_test = ner_dataset_test.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/17 [00:00<?, ? examples/s]

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForTokenClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN t

In [ ]:
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"{model_name}-finetuned-{task}-order",
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=5,
    weight_decay=0.01,
)

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = load_metric("seqeval")

<ipython-input-88-e20ba34f8cc7>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("seqeval")


In [ ]:
labels = [label_list[i] for i in example[f"{task}_tags_list"]]
labels

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORDER_ID',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
metric.compute(predictions=[labels], references=[labels])

{'overall_precision': 0.0,
 'overall_recall': 0.0,
 'overall_f1': 0.0,
 'overall_accuracy': 1.0}

In [ ]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
trainer = Trainer(
    model,
    args,
    train_dataset=tokenized_dataset_train,
    eval_dataset=tokenized_dataset_test,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [ ]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,No log,0.027958,0.000000,0.000000,0.000000,0.988796
2,No log,0.017835,0.000000,0.000000,0.000000,0.991597
3,No log,0.018735,0.000000,0.000000,0.000000,0.991597
4,No log,0.012962,0.000000,0.000000,0.000000,0.997199
5,No log,0.012865,0.000000,0.000000,0.000000,0.997199


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ORDER_ID seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 due to no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:495: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:189: RuntimeWarning: invalid value encountered in

TrainOutput(global_step=190, training_loss=0.01582365161494205, metrics={'train_runtime': 424.1771, 'train_samples_per_second': 1.768, 'train_steps_per_second': 0.448, 'total_flos': 16276007863752.0, 'train_loss': 0.01582365161494205, 'epoch': 5.0})

In [ ]:
# save model
trainer.save_model("drive/MyDrive/WhereOrderBot/model/ner_pretrained_order")

In [ ]:
# load model
from transformers import AutoModelForTokenClassification

my_model = AutoModelForTokenClassification.from_pretrained("drive/MyDrive/WhereOrderBot/model/ner_pretrained_order")

tokenizer = AutoTokenizer.from_pretrained("drive/MyDrive/WhereOrderBot/model/ner_pretrained_order")




In [ ]:
input_text = "Your input text goes here. 123434"
encoded_input = tokenizer.encode_plus(input_text, padding=True, truncation=True, return_tensors="pt")


In [ ]:
import torch
with torch.no_grad():
    logits = model(**encoded_input)[0]
    predicted_labels = torch.argmax(logits, dim=2).squeeze().tolist()

    print(predicted_labels)

[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]


In [ ]:
predicted = [label_list[label] for label in predicted_labels]
predicted

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'ORDER_ID',
 'ORDER_ID',
 'ORDER_ID',
 'ORDER_ID',
 'ORDER_ID']

In [ ]:
from transformers import pipeline

ner = pipeline("ner", model="drive/MyDrive/WhereOrderBot/model/ner_pretrained_order")


In [ ]:
prediction = ner("I have order with #123412 (123123) AASDAS #ASDAS")
prediction

[{'entity': 'LABEL_0',
  'score': 0.99819344,
  'index': 1,
  'word': 'i',
  'start': 0,
  'end': 1},
 {'entity': 'LABEL_0',
  'score': 0.9986578,
  'index': 2,
  'word': 'have',
  'start': 2,
  'end': 6},
 {'entity': 'LABEL_0',
  'score': 0.9980149,
  'index': 3,
  'word': 'order',
  'start': 7,
  'end': 12},
 {'entity': 'LABEL_0',
  'score': 0.9939719,
  'index': 4,
  'word': 'with',
  'start': 13,
  'end': 17},
 {'entity': 'LABEL_1',
  'score': 0.9913441,
  'index': 5,
  'word': '#',
  'start': 18,
  'end': 19},
 {'entity': 'LABEL_1',
  'score': 0.99802554,
  'index': 6,
  'word': '123',
  'start': 19,
  'end': 22},
 {'entity': 'LABEL_1',
  'score': 0.99832183,
  'index': 7,
  'word': '##41',
  'start': 22,
  'end': 24},
 {'entity': 'LABEL_1',
  'score': 0.9981902,
  'index': 8,
  'word': '##2',
  'start': 24,
  'end': 25},
 {'entity': 'LABEL_1',
  'score': 0.9969983,
  'index': 9,
  'word': '(',
  'start': 26,
  'end': 27},
 {'entity': 'LABEL_1',
  'score': 0.99812216,
  'index': 1